# 1.5深度学习实践活动

## 一、实践活动
1. 体验卷积神经网络模型训练的一般过程。
2. 利用训练好的模型，对新数据进行预测。

### 知识链接：
- 卷积神经网络模型

## 二、实验准备
- 已经预处理并分类的包含石头剪刀布图片的三个文件夹

### 知识链接：
- 数据增强(data_Augmentation)：Keras的ImageDataGenerator函数

## 三、实验步骤
### 第一步，导入相关库，并进行设置

In [11]:
import tensorflow as tf
from keras_preprocessing.image import ImageDataGenerator
from keras_preprocessing import image
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dropout, Dense
class_names = ['rock', 'scissors', 'paper']
class_names_chn = ['石头', '剪刀', '布']
img_width, img_height = 300, 200

### 第二步，构造卷积神经网络模型，进行石头、剪刀、布三分类

In [ ]:
def train_rps():
    
    # 构造图片读取的实例
    train_data_gen = ImageDataGenerator(
        rescale=1. / 255,
        rotation_range=40,
        width_shift_range=0.3,
        height_shift_range=0.3,
        shear_range=0.3,
        zoom_range=0.3,
        horizontal_flip=True,
        fill_mode='nearest')
    test_data_gen = ImageDataGenerator(rescale=1. / 255)
    
    # 训练数据集和测试数据集直接从相应文件夹获取，
    # 需要注意的是，不同的图片要访问相应的分类文件夹里
    # 如代表 石头 的图片应该放在 rock/子文件夹里
    train_datas = train_data_gen.flow_from_directory(
        "data/rsp/",
        target_size=(img_width, img_height),
        class_mode='categorical',
        classes=class_names
    )
    test_datas = test_data_gen.flow_from_directory(
        "data/rsp-test/",
        target_size=(img_width, img_height),
        class_mode='categorical',
        classes=class_names
    )
    # 卷积神经网络模型，可根据训练结果进行相应调参
    model = tf.keras.models.Sequential([
        Conv2D(64, (3, 3), activation='relu', input_shape=(img_width, img_height, 3)),
        MaxPooling2D(2, 2),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D(2, 2),
        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D(2, 2),
        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D(2, 2),
        Flatten(),
        Dropout(0.5),
        Dense(512, activation='relu'),
        Dense(3, activation='softmax')
    ])
    model.summary()
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    history = model.fit(train_datas, epochs=25, validation_data=test_datas, verbose=1)
    # 保存训练好的模型，下次就不用再训练了
    model.save("rsp.h5")

# 第一次运行可以先进行训练，下次就可直接使用
train_rps()

### 第三步，导入训练好的模型进行预测

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
# 选择需要识别的图片
# 用摄像头或者手机拍摄图片，如果没有的话直接找一张图片即可
img = image.load_img('rock.jpg', target_size=(img_height, img_width))
plt.figure('Image Show', dpi=300)
plt.axis('off')
plt.imshow(img)
x = image.img_to_array(img)
x = x/255.0
x = np.expand_dims(x, axis=0)
# pil_im = (255-pil_im)/255.0
# pil_im = pil_im.reshape(-1, img_width, img_height,3)/255.0
# 导入训练好的模型
model = tf.keras.models.load_model('rsp.h5')
pred = model.predict(x)
pred_label = np.argmax(pred[0])
print(f"预测为: {class_names_chn[int(pred_label)]}")

## 思考与讨论
1. 试改变剪刀石头布不同目录下的图片数量，是否对模型的准确度造成影响？
2. 修改卷积神经网络的不同参数，使得模型准确度更高。